In [72]:
from pymongo import MongoClient
from collections import Counter

In [73]:
uri = "mongodb://localhost:27017"
db_name = "mtg_decks_db"
client = MongoClient(uri)
db = client[db_name]

In [74]:
pipeline = [
    {"$match": {"isSanitized": True}},
    {"$project": {"all_cards": {"$concatArrays": [
        "$maindeck"]}, "name": 1}},
        # "$maindeck", "$sideboard"]}, "name": 1}},
    {"$unwind": "$all_cards"},
    {"$group": {"_id": {"_id": "$_id", "card_name": "$all_cards",
                        "name": "$name"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id._id", "name": {"$first": "$_id.name"},
                "cards": {"$push": {"name": "$_id.card_name", "count": "$count"}}, "total_cards": {"$sum": "$count"}}}
]

decks = list(db.decks.aggregate(pipeline))
decks

[{'_id': ObjectId('660e28e2990416447862219d'),
  'name': 'Walls Combo',
  'cards': [{'name': 'Crashing Drawbridge', 'count': 1},
   {'name': 'Llanowar Elves', 'count': 1},
   {'name': "Avacyn's Pilgrim", 'count': 1},
   {'name': 'Wall of Roots', 'count': 1},
   {'name': 'Shield-Wall Sentinel', 'count': 4},
   {'name': 'Drift of Phantasms', 'count': 4},
   {'name': 'Secret Door', 'count': 1},
   {'name': 'Freed from the Real', 'count': 1},
   {'name': 'Saruli Caretaker', 'count': 4},
   {'name': 'Winding Way', 'count': 4},
   {'name': 'Jaspera Sentinel', 'count': 2},
   {'name': 'Tuktuk Rubblefort', 'count': 2},
   {'name': 'Overgrown Battlement', 'count': 4},
   {'name': 'Valakut Invoker', 'count': 1},
   {'name': 'Lead the Stampede', 'count': 4},
   {'name': 'Axebane Guardian', 'count': 4},
   {'name': 'Quirion Ranger', 'count': 4},
   {'name': 'Forest', 'count': 13},
   {'name': 'Galvanic Alchemist', 'count': 1},
   {'name': 'Reaping the Graves', 'count': 1},
   {'name': 'Orochi Leaf

In [75]:
personal_pool = {}
for card in db.personal_pool.find({}, {"name": 1, "count": 1, "_id": 0}):
    personal_pool[card["name"]] = card["count"]

personal_pool

{'Forest': 59,
 'Island': 56,
 'Mountain': 58,
 'Plains': 60,
 'Snow-Covered Forest': 16,
 'Snow-Covered Island': 15,
 'Snow-Covered Mountain': 15,
 'Snow-Covered Plains': 2,
 'Snow-Covered Swamp': 1,
 'Swamp': 47,
 'Alms of the Vein': 4,
 'Arms of Hadar': 4,
 'Blood Celebrant': 4,
 'Blood Fountain': 4,
 'Cabal Ritual': 4,
 'Cast Down': 4,
 'Crypt Incursion': 2,
 'Crypt Rats': 4,
 'Crypt Sliver': 4,
 'Dark Ritual': 4,
 'Darkness': 4,
 'Deadly Dispute': 4,
 'Defile': 4,
 'Drown in Sorrow': 4,
 'Duress': 4,
 'Exhume': 4,
 'Extract a Confession': 3,
 'Faerie Macabre': 4,
 'Fanatical Offering': 4,
 'Feed the Swarm': 1,
 'Go for the Throat': 1,
 'Gray Merchant of Asphodel': 2,
 'Gurmag Angler': 4,
 'Hopeless Nightmare': 4,
 'Horror of the Broken Lands': 4,
 'Infectious Inquiry': 4,
 'Kitchen Imp': 4,
 'Lotleth Giant': 4,
 'Mukotai Ambusher': 2,
 'Okiba-Gang Shinobi': 3,
 'Omen of the Dead': 3,
 'Phyrexian Rager': 1,
 'Raven of Fell Omens': 4,
 'Reaping the Graves': 4,
 "Reckoner's Bargain":

In [76]:
min_deck_percentage = 0.93

missing_decks = []
for deck in decks:
    id = deck["_id"]
    name = deck["name"]
    total_cards = deck["total_cards"]
    cards = deck["cards"]
    
    cards_in_personal_pool = sum(min(card["count"], personal_pool.get(card["name"], 0)) for card in cards)
    deck_percentage = cards_in_personal_pool / total_cards
    
    if deck_percentage >= min_deck_percentage:
        missing_cards = [{"name": card["name"], "count": max(0, card["count"] - personal_pool.get(card["name"], 0))} for card in cards if personal_pool.get(card["name"], 0) < card["count"]]
        missing_decks.append({"id": id, "name": name, "missing_cards": missing_cards})
    

In [79]:
for deck in missing_decks:    
    if len(deck['missing_cards']) == 0:
        continue
    
    print(f"{deck['name']} ({deck['id']})")
    for card in deck["missing_cards"]:
        print(f"{card['count']} {card['name']}")
        # print(f"{card['name']}")
    print("\n")

Gruul Ponza (660e250f9904164478621f1e)
4 Thermokarst


Familiars (660e278799041644786220c4)
1 Step Through
1 Remove Soul
1 Merchant Scroll


Mono-Blue Faeries (660e268a9904164478622021)
2 Bonesplitter
1 Vapor Snag


Kuldotha Red (660e271d990416447862207f)
1 Goblin Sledder


Kuldotha Red (660e2c5299041644786223b8)
4 Goblin Javelineer


Mono-Blue Faeries (660e24459904164478621e8d)
4 Vapor Snag


Golgari Dredge (6614a550bc0fae7824b1c61d)
4 Mire Triton


Familiars (660e27259904164478622085)
2 Merchant Scroll


Mono-Blue Delver (660e2694990416447862202a)
2 Boomerang


Kuldotha Red (660e24e19904164478621efb)
2 Sawblade Scamp


Izzet Skred (6614a7eebc0fae7824b1c8c6)
3 Accumulated Knowledge


Orzhov Blade (660e274c990416447862209e)
2 Indoctrination Attendant


Golgari Dredge (6614a8bdbc0fae7824b1c9c4)
4 Mire Triton


Tron Ephemerate (6614a792bc0fae7824b1c867)
1 Runaway Boulder
1 Remote Isle


Boros Synthesizer (66169c122eb712be577eedea)
2 Rites of Initiation
1 Lorehold Campus


Boros Synthesiz